In [0]:
dbutils.fs.ls("/Volumes/big_data_ii_2025/spark_examples/checkpoints")

# Spark Structured Streaming

En el presente notebook veremos algunos ejemplos sobre Spark Structured Streaming.

Iniciamos creando un stream con un _rate source_ - referencia: [RateStreamSource](https://jaceklaskowski.gitbooks.io/spark-structured-streaming/content/spark-sql-streaming-RateStreamSource.html) 

In [0]:
rateDF = spark.readStream.format("rate")\
          .option("rowsPerSecond",10)\
          .load()

El siguiente paso es generar una agregacion por cada 10 segundos para mostrar las ventanas y los _micro-batches_.

In [0]:
from pyspark.sql.functions import window
agg = rateDF.groupBy(window("timestamp","15 seconds")).count()


Finalmente la ejecutamos y vemos la salida a la consola:

In [0]:
#NOTA: DAtabricks Free Edition no permite la ejecución de streaming con triggers por tiempo, por lo que el siguiente código genera un error:
q = agg.writeStream.outputMode("complete")\
      .format("console").start()

Pero permite la ejecución de streaming con triggers de tipo _AvailableNow_ o _Once_ ... o sin triggers.


In [0]:
q = (agg.writeStream
         .outputMode("complete")     
         .format("console")
         .trigger(availableNow=True)         
         .option("checkpointLocation", "/Volumes/big_data_ii_2025/spark_examples/checkpoints")
         .start())

In [0]:
# Ver el status:
print(q.status)

In [0]:
#Metricas:
display(q.recentProgress)

In [0]:
#Espera 2 minutos para que termine el job, sino hace un timeout.
#q.awaitTermination(120)

Para detenerlo:

In [0]:
q.stop()